初始状态下，点击所有位置Reward暂定为0

In [52]:
from MineSweeper.core import MineSweeperService,MineSweeperGame,Cell,CellType
import hashlib

class RewardSingleData:
    def __init__(self, curGame : MineSweeperGame, clickX, clickY,reward = 0):
        self.curGame = curGame
        self.clickX = clickX
        self.clickY = clickY
        self.reward = reward
    
    def GetHashValue(self):
        return calculate_string_hash(self.curGame.GetGameDesString()+str(self.clickX)+str(self.clickY))

class RewardValueClass:
    def __init__(self,curRewardValue):
        self.curRewardValue = curRewardValue
        self.Count = 1

    def CalculateAverage(self,newRewardValue):
        self.curRewardValue = float(self.curRewardValue*self.Count + newRewardValue)/float(self.Count+1)
        self.Count += 1

class RewardData:
    def __init__(self):
        self.rewardDataDict = {}
    
    def AddRewardSingleData(self,rewardSingleData):
        curGameHashValue = rewardSingleData.GetHashValue()
        if self.rewardDataDict.get(curGameHashValue,None) is None:
            self.rewardDataDict.setdefault(curGameHashValue,RewardValueClass(rewardSingleData.reward))
        else:
            self.rewardDataDict[curGameHashValue].CalculateAverage(float(rewardSingleData.reward))
        
    def GetRewardValueDataCurRewardValue(self,rewardSingleData:RewardSingleData):
        curGameHashValue = rewardSingleData.GetHashValue()
        rewardValueClass : RewardValueClass = self.rewardDataDict.get(curGameHashValue,None)
        if  rewardValueClass is None:
            return False,None
        else:
            return True,rewardValueClass.curRewardValue

    def ClearRewardData(self):
        self.rewardDataDict.clear()

def calculate_string_hash(s):
    # 创建一个哈希对象
    hash_obj = hashlib.new("sha256")
    # 更新哈希对象的内容
    hash_obj.update(s.encode("utf-8"))
    # 返回十六进制格式的哈希值
    return hash_obj.hexdigest()


In [53]:
mineSweeperService = MineSweeperService(height=3,width = 3,mineNum = 3)
rewardData = RewardData()
iter = 100000
for _ in range(iter):
    if mineSweeperService.IsGameOver():
        mineSweeperService.ReSetCurGame()
    mineSweeperService.RandomSafeClick()
    curGame = mineSweeperService.GetCurGame()
    for unRevealedCellIndex in curGame.UnRevealedSet:
        unRevealedCell = curGame.cellGrid.cells[unRevealedCellIndex[0]][unRevealedCellIndex[1]]
        unRevealedCell : Cell
        if unRevealedCell.cellType == CellType.Mine:
            reward = -2
        else:
            reward = 1
        rewardSingleData = RewardSingleData(curGame=curGame,
                                            clickX= unRevealedCell.pos[0],
                                            clickY=unRevealedCell.pos[1],
                                            reward=reward)
        rewardData.AddRewardSingleData(rewardSingleData=rewardSingleData)

In [54]:
len(rewardData.rewardDataDict)

33398

In [55]:
for rewarddataDictValue in rewardData.rewardDataDict.values():
    print(rewarddataDictValue.curRewardValue)

-0.20042949176807445
0.007874015748031494
-0.08231925554760203
-0.7329992841803867
0.07444523979957053
-0.0007158196134574482
-0.026485325697924143
-0.03937007874015749
-0.4999999999999999
0.07857142857142857
0.1
-0.3285714285714283
-0.17857142857142858
-0.09285714285714286
-1.0785714285714285
-1.1428571428571428
-0.14285714285714285
-1.0
-0.8571428571428571
1.0
-0.8571428571428571
-2.0
-2.0
-2.0
1.0
1.0
-2.0
-2.0
1.0
1.0
-0.5712290502793288
-1.1201117318435747
0.660614525139665
0.5474860335195522
-1.3086592178770957
0.6270949720670385
0.7025139664804466
-0.5377094972067049
-2.0
1.0
1.0
-2.0
1.0
1.0
-2.0
-2.0
1.0
1.0
-2.0
1.0
-2.0
-2.0
1.0
1.0
-2.0
1.0
-0.14990512333965847
-0.37760910815939314
0.24478178368121442
-0.13662239089184075
-0.5426944971537015
-0.10246679316888045
-0.23339658444022762
0.2979127134724857
0.09210526315789473
-0.34210526315789486
-0.5789473684210525
0.46710526315789475
-0.9539473684210528
-0.5592105263157896
-0.125
0.14285714285714285
-0.9285714285714286
-1.1428

In [56]:
for rewarddataDictValue in rewardData.rewardDataDict.values():
    print(rewarddataDictValue.Count)

1397
1397
1397
1397
1397
1397
1397
1397
140
140
140
140
140
140
140
42
42
42
42
42
42
15
15
15
15
15
5
5
6
6
716
716
716
716
716
716
716
716
24
24
24
24
24
24
24
14
14
14
14
14
14
5
6
6
5
6
1581
1581
1581
1581
1581
1581
1581
1581
152
152
152
152
152
152
152
28
28
28
28
28
28
2
3
3
3
3
449
449
449
449
449
449
449
449
44
44
44
44
44
44
44
2
3
3
3
3
3
129
129
129
129
129
129
129
39
39
39
39
39
39
21
21
21
21
21
17
17
17
17
5
8
4
1413
1413
1413
1413
1413
1413
1413
1413
49
49
49
49
49
49
49
9
9
9
9
9
9
7
7
7
7
7
16
16
16
16
8
6
7
5
5
5
4
5
4
5
620
620
620
620
620
620
620
620
140
140
140
140
140
140
140
7
6
7
7
4
7
1631
1631
1631
1631
1631
1631
1631
1631
155
155
155
155
155
155
155
19
19
19
19
19
19
22
22
22
22
22
14
14
14
14
48
48
48
123
123
123
123
123
123
123
9
6
9
9
6
9
5
5
5
5
3
5
5
279
279
279
279
279
279
279
279
105
105
105
105
105
105
105
2
2
1
2
2
2
1488
1488
1488
1488
1488
1488
1488
1488
80
80
80
80
80
80
80
9
9
9
9
9
9
14
14
14
14
14
93
93
93
93
26
34
20
2770
2770
2770
2770
2770
2

In [57]:
print(mineSweeperService.GetCurGameDesString())

B2B
BB2
2BB



In [58]:
curGameReward = RewardSingleData(mineSweeperService.GetCurGame(),0,1,0)
_,rewardValue= rewardData.GetRewardValueDataCurRewardValue(curGameReward)
print(rewardValue)

None


In [59]:
mineSweeperService.ReSetCurGame()

In [60]:
mineSweeperService.RandomSafeClick()

In [61]:
def RewardPWinRate(mineSweeperService:MineSweeperService,iter):
    GameRun = 0
    WinRun =0
    LossRun = 0
    
    for _ in range(iter):
        if mineSweeperService.IsGameStart():
            mineSweeperService.RandomClick()
        curGame = mineSweeperService.GetCurGame()

        curMaxReward = -2
        for pos in curGame.UnRevealedSet:
            clickX = pos[0]
            clickY = pos[1]
            curRewardSingleData = RewardSingleData(mineSweeperService.GetCurGame(),clickX,clickY,0)
            vaild,rewardValue = rewardData.GetRewardValueDataCurRewardValue(curRewardSingleData)
            if vaild:
                if rewardValue >= curMaxReward:
                    maxClickX = clickX
                    maxClickY = clickY
                    curMaxReward = rewardValue
            else:
                print("Never meet curGame")
                print(curGame.GetGameDesString())
                print(clickX,clickY)
                maxClickX = clickX
                maxClickY = clickY
                break
        mineSweeperService.TryClick(maxClickX,maxClickY)
        if mineSweeperService.IsGameOver():
            if mineSweeperService.IsGameWin():
                WinRun += 1
            if mineSweeperService.IsGameLoss():
                LossRun += 1
            
            GameRun += 1
            mineSweeperService.ReSetCurGame()
            

    print(f"WinRun:{WinRun},GameRun:{GameRun},LossRun:{LossRun}")
    print("WinRate:",float(WinRun) / float(GameRun))

In [63]:
RewardPWinRate(mineSweeperService,100000)

WinRun:17589,GameRun:32197,LossRun:14608
WinRate: 0.5462931329005808
